## Mapping soil pH variability in the Zimbabwe Sugar Industry:

In [44]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt

: 

In [45]:
data_2018 = pd.read_excel("data/Estates Growers & ZSAES Soil 2018.xls")
data_2018['pH (CaCl2)'] = data_2018['pH'].astype(float)

### starting with 2018 data

In [ ]:
data_2018.head()

In [ ]:
data_2018.describe()

In [ ]:
print(data_2018.isna().sum())

In [ ]:
# dropp non estate data and null data
data_2018.columns = data_2018.columns.str.strip()
data_2018_curated = data_2018.dropna(subset=['pH', 'Section'])
cats = []
for obs in data_2018['Lab Ref']:
    if(str(obs).startswith('TSS')):
        cats.append('TRI-Private')
    elif(str(obs).startswith('HSS')):
        cats.append('HVE-Private')
    elif(str(obs).startswith('MSS')):
        cats.append('MKW-Private')
    elif(str(obs).startswith('ZS')):
        cats.append('ZSAES')
    elif(str(obs).startswith('HS')):
        cats.append('HVE')
    elif(str(obs).startswith('MS')):
        cats.append('MKW')
    elif(str(obs).startswith('TS')):
        cats.append('TRI')
    else:
        cats.append('Private')
        
data_2018['zone'] = cats
data_2018.zone.value_counts()

In [ ]:
data = data_2018[data_2018['zone'] != 'Private']

data['zone_1'] = data['zone'] + ' - ' + data['Section'].astype(str)
data.head()

In [ ]:
# Group by zone_1 and compute mean, min, and max pH
grouped_2 = data.groupby('zone_1')['pH'].agg(['mean', 'min', 'max']).reset_index()

# Find the corresponding farm/section for min and max pH in each zone_1
sect_pH_min = data.loc[data.groupby('zone_1')['pH'].idxmin(), ['zone_1', 'Field #']]
sect_pH_max = data.loc[data.groupby('zone_1')['pH'].idxmax(), ['zone_1', 'Field #']]

# Rename columns for clarity
sect_pH_min = sect_pH_min.rename(columns={'Field #': 'sect_pH_min'})
sect_pH_max = sect_pH_max.rename(columns={'Field #': 'sect_pH_max'})

# Merge back with grouped data
grouped_2 = grouped_2.merge(sect_pH_min, on='zone_1', how='left')
grouped_2 = grouped_2.merge(sect_pH_max, on='zone_1', how='left')

print(grouped_2)
